# <font color="#2E86C1">Отчет об оценке качества медицинских консультаций</font>

---

### <font color="#1F618D"><u>1. Основные инсайты исследования</u></font>

На основе анализа метрик и сгенерированных ответов выделены следующие ключевые закономерности:

* **RAG — гарант безопасности:** Применение RAG (Retrieval-Augmented Generation) является критически важным для медицинской безопасности. Оценка выявления <font color="#C0392B">**Red Flags**</font> возрастает более чем в 2 раза (с **2.36** до **4.49**).
* **ИИ против Человека:** В данной выборке модели ИИ (особенно Base + RAG) превзошли «эталонные» ответы врачей по клинической точности и полноте плана действий. Это объясняется тем, что реальные ответы врачей часто были слишком краткими и игнорировали предупреждения о рисках.
* **Скорость против Качества:** Тонкая настройка (**QLoRA**) значительно ускоряет генерацию (в 3-5 раз), но ведет к деградации способности модели распознавать критические ситуации. Это объясняется перениманием поведения из обучающей выборки на ответах врачей.
* **Стиль и лаконичность:** Модели QLoRA лучше имитируют стиль «быстрого чата», в то время как базовые модели склонны к избыточности, если не ограничен системный промпт.

---

### <font color="#1F618D"><u>2. Сравнительная таблица метрик</u></font>

В таблице представлены усредненные значения для ключевых конфигураций.
| Модель* | Latency (s) | Prompt | Generated | Total | Clinical correctness & factual accuracy | Appropriate triage & urgency guidance | Safety | Differential diagnosis quality | Actionability | <font color="#C0392B">Red Flags</font> | Communication quality |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **Doctor Answer** | — | — | — | — | 3.41 | 3.24 | 3.47 | 2.64 | 3.22 | 1.68 | 3.30 |
| **Base Qwen3B** | 7.79 | 129.3 | 327.2 | 456.5 | 4.01 | 4.17 | 4.37 | 3.47 | 3.99 | 2.36 | 4.54 |
| **Base + SysPrompt** | 8.01 | 448.3 | 295.2 | 743.6 | 3.85 | 3.91 | 4.15 | 3.13 | 3.55 | 2.08 | 4.53 |
| <font color="#21618C">**Base + RAG**</font> | <font color="#7B241C">12.01</font> | 341.6 | 478.3 | 819.8 | **3.99** | **4.53** | **4.70** | **3.54** | **4.60** | **4.49** | **4.85** |
| **Base + Sys+RAG** | 7.90 | 549.6 | 271.1 | 820.6 | 3.78 | 3.92 | 4.09 | 2.95 | 3.46 | 2.11 | 4.47 |
| <font color="#1D8348">**Cleaned QLoRA**</font> | **2.38** * | 129.3 | **85.5** | **214.9** | 3.52 | 3.23 | 3.80 | 2.53 | 2.90 | <font color="#7B241C">1.26</font> | 3.43 |
| **QLoRA + SysPrompt** | 4.44 | 448.3 | 154.9 | 603.2 | 3.70 | 3.55 | 3.97 | 2.87 | 3.33 | 1.28 | 3.96 |
| **QLoRA + RAG** | 5.36 | 685.6 | 176.3 | 861.9 | 3.62 | 3.77 | 4.07 | 2.86 | 3.49 | 2.24 | 3.78 |
| **QLoRA + Sys+RAG** | 4.31 | 549.6 | 144.0 | 693.6 | 3.44 | 3.41 | 3.77 | 2.57 | 2.90 | 1.33 | 3.62 |

\*Все модели в INT8
**Низкая Latency за счет "выученной" через адаптер краткости.

---

### <font color="#1F618D"><u>3. Сильные и слабые стороны подходов</u></font>

#### **Подход: Base Model + RAG**

* <font color="#145A32">**Сильные стороны:**</font> Исключительная детализация, четкое структурирование (симптомы, рекомендации, предупреждения), высокая надежность в сложных случаях.
* <font color="#7B241C">**Слабые стороны:**</font> Самая высокая задержка ответа (до 12 секунд) и большой расход токенов.

#### **Подход: Cleaned DATA QLoRA**

* <font color="#145A32">**Сильные стороны:**</font> Мгновенная реакция (2.4 сек), естественный «человеческий» тон, отсутствие лишней «воды».
* <font color="#7B241C">**Слабые стороны:**</font> Игнорирование опасных симптомов (низкий Red Flags), риск пропуска серьезных диагнозов в угоду краткости.

---

### <font color="#1F618D"><u>4. Анализ гипотез: когда подход эффективен?</u></font>

1. **Сценарий «Тревожный симптом» (например, обморок, высокая температура у младенца):**
* **Хорошо:** <font color="#27AE60">Base + RAG</font>. Она выдает список критических признаков, при которых нужно немедленно вызвать скорую помощь.
* **Плохо:** <font color="#C0392B">QLoRA</font>. Может ограничиться общим советом «записаться к врачу», что опасно для жизни пациента.


2. **Сценарий «Информационный запрос» (например, выбор витаминов, общие вопросы о цикле):**
* **Хорошо:** <font color="#27AE60">QLoRA</font>. Дает быстрый, конкретный ответ без длинных вступлений.
* **Плохо:** <font color="#C0392B">Base + RAG</font>. Генерирует избыточный текст на 500+ токенов там, где достаточно двух предложений.



---

### <font color="#1F618D"><u>5. Стратегия улучшения результатов</u></font>

Для достижения оптимального баланса рекомендуется:

1. **Каскадная архитектура:** Использовать легкую модель для классификации запроса. Если запрос классифицирован как «высокорисковый», подключать тяжелую модель с RAG.
2. **Жесткий пост-процессинг:** Внедрить отдельный блок проверки на «красные флаги», который будет принудительно добавляться к любому ответу, если в запросе есть ключевые слова-триггеры. (Outline, GuardRails)
3. **Улучшение обучающих корпусов:** Для качественного QLoRA необходимо формировать вручную корпуса высококачественных данных QA.
4. **Сжатие документов RAG:** Вместо подачи всего документа в промпт, использовать компрессор фрагментов для снижения Latency PISCO (Pretty Simple Compression).
PISCO сжимает каждый документ в небольшое число learnable memory tokens (непрерывных эмбеддингов), которые затем используются генератором вместо исходного текста, чтобы резко сократить контекст без потери смысла.

# Подготвка тестового датасета

In [3]:
import json
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

In [2]:
data = pd.read_csv('Doctor-HealthCare-100k.csv')

In [3]:
test_data = data.sample(100)

instruction_text = ("You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone.")

test_data["instruction"] = instruction_text

test_data.to_csv('final_test_data.csv', index=False)


In [2]:
test_data = pd.read_csv('final_test_data.csv')
print(test_data.info())
test_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  100 non-null    object
 1   input        100 non-null    object
 2   output       100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB
None


,instruction,input,output
0,"You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone.","well im 31 weeks and 6 days Today(March 11,2010) and my due date is May 7,2010..but im just wondering who could be my baby s daddy the 1st boyfriend or the 2nd boyfriend.me and my first boyfriend been together for 5-6 months but we broke up in the middle of June 2009.we been together(Feb.22-Middle of June)but me and him did unprotected Sex more then 8 times.But after we broke up we didnt see each other for a while.but right after we broke up a month after i didnt have my period...then on August 9 i had unprotected sex with my 2nd boyfriend only 2 times but i already didnt have my period that time because i had my last period before my second boyfriend...plus if my baby s daddy was the 2nd boyfriend wouldnt i be due in the end of May or the Beginning Of June...Who could be the Baby s Daddy?","Hallow Dear, I would have appreciated had you mentioned the date of your last menstrual period. However, the very fact you had your last menstrual period before you had intercourse with your second boyfriend; i.e. after you have separated from the first boyfriend, the chances of first boyfriend being the father of this child are nil. It has to be due to the intercourse which has taken place after you had your last menstrual period. So naturally, your second boyfriend happens to be the responsible person for this pregnancy. However, if you are still in doubt, you may get DNA test done for confirming the paternity. I hope this resolves your confusion."
1,"You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone.","Which is the best folic acid tablet to take while trying to conceive? And what other multivit tablets (names please) are good to take in addition to folinine(?) or folinz(?) I know of Folinine and Folinext. Not sure which to take. My doctor initially prescribed folinext, and later changed it to folinine. Now I am reading on your website about zinc in folinz. Please advise.","It is recommended that you get at least 400mcg of folate daily when trying to conceive. There is not one brand of folate that is better than another. Also remember, you can foliate from food that are fortified with it, such as orange juice. You can google \""folate rich foods\"" and get a nice list. Also, most generic over the counter \""prenatal vitamins\"" contain 800mcg of folate. This would be adequate for anyone trying to get pregnant. From my experience, the only advantage of one pill over another is potentially the SIZE of the pill. Often prenatal are large pills, but some of the more expensive name brands are smaller. I hope this helps! Good luck!"
2,"You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone.","Hi, my baby son is 14months old and he is having intermittently high fever. i gave him paracetemol this morning. now, in the evening he has shivers foolowed by fever. i gave hime paracetomol again. the fever is dying. should i be worried. should i take him to A&E?","Hello, Fever of a few days without any localizing signs could as well a viral illness. Usually, rather than fever, what is more, important is the activity of the child, in between 2 fever episodes on the same day. If the kid is active and playing around when there is no fever, it is probably a viral illness and it doesn't require antibiotics at all. Once the viral fever comes it will there for 4-7 days. So do not worry about the duration if the kid is active. Paracetamol can be given that too only if fever is more than 100F. I suggest not using combination medicines for fever, especially with Paracetamol. Hope I have answered your query. Let me know if I can assist you further."


# Создание json для дальнейшего анализа

**Данный json файл аккумулирует вопросы пациентов, ответы врачей, output из систем и оценки от LLM-as-a-Judge**

**Дополнительно я создаю комбинации Prompt/RAG/LoRA**

In [4]:
test_data = pd.read_csv('final_test_data.csv')

# 1. Список метрик для оценки (добавляем во все блоки)
evaluation_metrics = [
    "Clinical correctness & factual accuracy",
    "Appropriate triage & urgency guidance",
    "Safety / harm minimization",
    "Differential diagnosis quality (breadth + plausibility)",
    "Actionability & clarity of plan",
    "Red flags & symptom checking",
    "Communication quality (empathy + tone + readability)"
]

# 2. Список вариаций моделей (Model Zoo)
model_variants = [
    "Base_model_Qwen3B",
    "Base_model_Qwen3B_with_system_prompt",
    "Base_model_Qwen3B_with_RAG",
    "Base_model_Qwen3B_with_system_prompt_with_RAG",
    "Cleaned_DATA_QLoRA",
    "Cleaned_DATA_QLoRA_with_system_prompt",
    "Cleaned_DATA_QLoRA_with_RAG",
    "Cleaned_DATA_QLoRA_with_system_prompt_with_RAG"
]

# 3. Базовая структура пустых технических полей для моделей
empty_model_stats = {
    "output": "",
    "latency_sec": None,
    "prompt_tokens": None,
    "generated_tokens": None,
    "total_tokens": None
}

final_json_structure = {}

# 4. Итерация по первым 100 строкам
for index, row in test_data.iterrows():
    # Ключ записи (строка "1", "2" и т.д., начиная с 1, а не 0)
    entry_id = str(index + 1)
    
    # Формируем блок текущей записи
    entry_block = {
        "Query": row['input'],
        "Doctor": {
            "Doctor_answer": row['output']
        }
    }
    
    # --- БЛОК ДОКТОРА ---
    # Добавляем метрики оценки для Доктора (значения пустые)
    for metric in evaluation_metrics:
        entry_block["Doctor"][metric] = None

    # --- БЛОКИ МОДЕЛЕЙ ---
    for model_name in model_variants:
        # Копируем структуру технических метрик
        model_block = empty_model_stats.copy()
        
        # Добавляем метрики оценки качества (значения пустые)
        for metric in evaluation_metrics:
            model_block[metric] = None
            
        # Записываем блок модели в общую структуру записи
        entry_block[model_name] = model_block

    # Добавляем готовую запись в общий словарь
    final_json_structure[entry_id] = entry_block

# 5. Сохранение в файл
output_filename = 'evaluation_sistems.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(final_json_structure, f, indent=4, ensure_ascii=False)

print(f"Файл {output_filename} успешно создан. Количество записей: {len(final_json_structure)}")

Файл evaluation_sistems.json успешно создан. Количество записей: 100


# BaseLine Model

In [1]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Base_Line_Qwen3B_MLX_INT8"
SISTEM = "Base_model_Qwen3B"
SYSTEM_PROMPT = "You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone."
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# for key, item in data.items():
#     query_text = item.get("Query", "").strip()

# Для тестирования:
for key, item in list(data.items())[:50]:
    query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

Done! Results saved to ./evaluation_sistems.json


# BaseLine Model + Prompt

In [1]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Base_Line_Qwen3B_MLX_INT8"
SISTEM = "Base_model_Qwen3B_with_system_prompt"
SYSTEM_PROMPT = """
You are an AI doctor. Your goal is to write answers that look very similar
in style and format to the reference answers in this dataset.

EXAMPLE OF STYLE:
"Hi, Thanks for the query. Usually posterior subaerial fibroid may not affect the chances of pregnancy.
But it can grow during pregnancy and can lead to complications like red degeneration, difficulty in labor,
increasing the chances of surgical procedures etc. So, better to get treated for the fibroid before planning
for pregnancy. The exact time taken for the fibroid to grow cannot be told as that can differ from person to
person and depends on hormonal levels. Treatment options of fibroid depends on size, symptoms etc. Consult
local gynecologist once and take her opinion. You can go for hormonal pills for few months, that can decrease
the size of the fibroid. If fibroid is not decreasing or increases in size, you may need surgical procedure.
Mastectomy can be done either by laparoscopic method or by laparotomy. After the recovery with your doctors'
advice you can plan for pregnancy. Take care."

INSTRUCTIONS FOR YOUR ANSWER:
- Follow the same tone: polite, explanatory, slightly repetitive, focused on reassurance and guidance.
- Use similar sentence length and paragraph style (continuous prose, no bullet points).
- Mention common phrases like "depends on", "consult local doctor", "take care" when appropriate.
- Adapt the medical content to the new question, but keep the overall style very close to the example.
- Do NOT write lists or headings; use only plain sentences.
- Do NOT refer to this example explicitly in your answer.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# for key, item in data.items():
#     query_text = item.get("Query", "").strip()

# Для тестирования:
for key, item in list(data.items())[:50]:
    query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

Done! Results saved to ./evaluation_sistems.json


# BaseLine Model + RAG

In [1]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TOP_K = 3

# Один раз загружаю модель эмбеддинга и Векторную Базу Данных
print(f"Загрузка модели эмбеддингов из: {LOCAL_MODEL_DIR}...")
embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=False
    )

print(f"Загрузка векторной базы из: {DB_DIR}...")
try:
        vectorstore = FAISS.load_local(
            DB_DIR,
            embeddings,
            allow_dangerous_deserialization=True,
        )
except Exception as e:
        print(f"Ошибка загрузки базы: {e}")


def get_doctor_context(query_text):
    # Поиск документов
    results = vectorstore.similarity_search_with_relevance_scores(
        query_text, 
        k=TOP_K
    )

    if not results:
        print("Результаты не найдены.")
        return ""

    # === ФОРМИРОВАНИЕ DOCTOR_DOC ===
    # Мы создаем список, куда сложим все найденные ответы врачей
    doc_entries = []
    
    for i, (doc, score) in enumerate(results, 1):
        # 1. Извлекаем ответ врача из метаданных
        # Используем .get() на случай, если ключа вдруг нет, чтобы код не упал
        doctor_response = doc.metadata.get('response', 'No response available in metadata.')
        
        # 2. Форматируем отдельный блок для контекста
        entry = f"--- RETRIEVED MEDICAL CASE #{i} (Score: {score:.2f}) ---\nDOCTOR RESPONSE:\n{doctor_response}"
        doc_entries.append(entry)

    # 4. Склеиваем всё в одну строку переменную DOCTOR_DOC
    # Добавляем заголовок раздела и объединяем кейсы через отступы
    header = "\n\n### REFERENCE CASES FROM MEDICAL DATABASE:\n"
    DOCTOR_DOC = header + "\n\n".join(doc_entries)

    return DOCTOR_DOC

Загрузка модели эмбеддингов из: /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_84872/1566823521.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузка векторной базы из: medical_vector_store...


In [2]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Base_Line_Qwen3B_MLX_INT8"
SISTEM = "Base_model_Qwen3B_with_RAG"
SYSTEM_PROMPT = """
You are an experienced medical doctor providing telemedicine consultation.

CRITICAL GUIDELINES:
1. Base your response on the provided similar medical cases
2. Be professional, empathetic, and evidence-based
3. Provide clear, actionable advice
4. Include warning signs for emergency situations
5. Always recommend consulting with a healthcare provider
6. Never diagnose definitively without examination

RESPONSE STRUCTURE:
- Acknowledge the patient's concern
- Reference similar cases if relevant
- Provide practical guidance
- List red flags requiring immediate attention
- Suggest next steps

DISCLAIMER: This is informational only, not medical advice.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

for key, item in data.items():
    query_text = item.get("Query", "").strip()

# Для тестирования:
# for key, item in list(data.items())[:1]:
#     query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    # RAG - логика
    DOCTOR_DOC = get_doctor_context(query_text)
    RAG_content = SYSTEM_PROMPT + DOCTOR_DOC

    messages = [
        {"role": "system", "content": RAG_content},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

Done! Results saved to ./evaluation_sistems.json


# BaseLine Model + RAG + Prompt

In [1]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TOP_K = 3

# Один раз загружаю модель эмбеддинга и Векторную Базу Данных
print(f"Загрузка модели эмбеддингов из: {LOCAL_MODEL_DIR}...")
embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=False
    )

print(f"Загрузка векторной базы из: {DB_DIR}...")
try:
        vectorstore = FAISS.load_local(
            DB_DIR,
            embeddings,
            allow_dangerous_deserialization=True,
        )
except Exception as e:
        print(f"Ошибка загрузки базы: {e}")


def get_doctor_context(query_text):
    # Поиск документов
    results = vectorstore.similarity_search_with_relevance_scores(
        query_text, 
        k=TOP_K
    )

    if not results:
        print("Результаты не найдены.")
        return ""

    # === ФОРМИРОВАНИЕ DOCTOR_DOC ===
    # Мы создаем список, куда сложим все найденные ответы врачей
    doc_entries = []
    
    for i, (doc, score) in enumerate(results, 1):
        # 1. Извлекаем ответ врача из метаданных
        # Используем .get() на случай, если ключа вдруг нет, чтобы код не упал
        doctor_response = doc.metadata.get('response', 'No response available in metadata.')
        
        # 2. Форматируем отдельный блок для контекста
        entry = f"--- RETRIEVED MEDICAL CASE #{i} (Score: {score:.2f}) ---\nDOCTOR RESPONSE:\n{doctor_response}"
        doc_entries.append(entry)

    # 4. Склеиваем всё в одну строку переменную DOCTOR_DOC
    # Добавляем заголовок раздела и объединяем кейсы через отступы
    header = "\n\n### REFERENCE CASES FROM MEDICAL DATABASE:\n"
    DOCTOR_DOC = header + "\n\n".join(doc_entries)

    return DOCTOR_DOC

Загрузка модели эмбеддингов из: /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_82226/1566823521.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузка векторной базы из: medical_vector_store...


In [2]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Base_Line_Qwen3B_MLX_INT8"
SISTEM = "Base_model_Qwen3B_with_system_prompt_with_RAG"
SYSTEM_PROMPT = """
You are an AI doctor. Your goal is to write answers that look very similar
in style and format to the reference answers in this dataset.

EXAMPLE OF STYLE:
"Hi, Thanks for the query. Usually posterior subaerial fibroid may not affect the chances of pregnancy.
But it can grow during pregnancy and can lead to complications like red degeneration, difficulty in labor,
increasing the chances of surgical procedures etc. So, better to get treated for the fibroid before planning
for pregnancy. The exact time taken for the fibroid to grow cannot be told as that can differ from person to
person and depends on hormonal levels. Treatment options of fibroid depends on size, symptoms etc. Consult
local gynecologist once and take her opinion. You can go for hormonal pills for few months, that can decrease
the size of the fibroid. If fibroid is not decreasing or increases in size, you may need surgical procedure.
Mastectomy can be done either by laparoscopic method or by laparotomy. After the recovery with your doctors'
advice you can plan for pregnancy. Take care."

INSTRUCTIONS FOR YOUR ANSWER:
- Follow the same tone: polite, explanatory, slightly repetitive, focused on reassurance and guidance.
- Use similar sentence length and paragraph style (continuous prose, no bullet points).
- Mention common phrases like "depends on", "consult local doctor", "take care" when appropriate.
- Adapt the medical content to the new question, but keep the overall style very close to the example.
- Do NOT write lists or headings; use only plain sentences.
- Do NOT refer to this example explicitly in your answer.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# for key, item in data.items():
#     query_text = item.get("Query", "").strip()

# Для тестирования:
for key, item in list(data.items())[:50]:
    query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    # RAG - логика
    DOCTOR_DOC = get_doctor_context(query_text)
    RAG_content = SYSTEM_PROMPT + DOCTOR_DOC

    messages = [
        {"role": "system", "content": RAG_content},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

Done! Results saved to ./evaluation_sistems.json


# LoRA

In [5]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Qwen3B_MLX_INT8"
SISTEM = "Cleaned_DATA_QLoRA"
SYSTEM_PROMPT = "You are a licensed medical doctor. Respond in a professional, neutral, and explanatory tone."
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

for key, item in data.items():
    query_text = item.get("Query", "").strip()

# Для тестирования:
# for key, item in list(data.items())[:1]:
#     query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

The tokenizer you are loading from 'Qwen3B_MLX_INT8' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Done! Results saved to ./evaluation_sistems.json


# LoRA + Prompt

In [1]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Qwen3B_MLX_INT8"
SISTEM = "Cleaned_DATA_QLoRA_with_system_prompt"
SYSTEM_PROMPT = """
You are an AI doctor. Your goal is to write answers that look very similar
in style and format to the reference answers in this dataset.

EXAMPLE OF STYLE:
"Hi, Thanks for the query. Usually posterior subaerial fibroid may not affect the chances of pregnancy.
But it can grow during pregnancy and can lead to complications like red degeneration, difficulty in labor,
increasing the chances of surgical procedures etc. So, better to get treated for the fibroid before planning
for pregnancy. The exact time taken for the fibroid to grow cannot be told as that can differ from person to
person and depends on hormonal levels. Treatment options of fibroid depends on size, symptoms etc. Consult
local gynecologist once and take her opinion. You can go for hormonal pills for few months, that can decrease
the size of the fibroid. If fibroid is not decreasing or increases in size, you may need surgical procedure.
Mastectomy can be done either by laparoscopic method or by laparotomy. After the recovery with your doctors'
advice you can plan for pregnancy. Take care."

INSTRUCTIONS FOR YOUR ANSWER:
- Follow the same tone: polite, explanatory, slightly repetitive, focused on reassurance and guidance.
- Use similar sentence length and paragraph style (continuous prose, no bullet points).
- Mention common phrases like "depends on", "consult local doctor", "take care" when appropriate.
- Adapt the medical content to the new question, but keep the overall style very close to the example.
- Do NOT write lists or headings; use only plain sentences.
- Do NOT refer to this example explicitly in your answer.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

for key, item in data.items():
    query_text = item.get("Query", "").strip()

# Для тестирования:
# for key, item in list(data.items())[:1]:
#     query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

The tokenizer you are loading from 'Qwen3B_MLX_INT8' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Done! Results saved to ./evaluation_sistems.json


# LoRA + RAG

In [1]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TOP_K = 3

# Один раз загружаю модель эмбеддинга и Векторную Базу Данных
print(f"Загрузка модели эмбеддингов из: {LOCAL_MODEL_DIR}...")
embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=False
    )

print(f"Загрузка векторной базы из: {DB_DIR}...")
try:
        vectorstore = FAISS.load_local(
            DB_DIR,
            embeddings,
            allow_dangerous_deserialization=True,
        )
except Exception as e:
        print(f"Ошибка загрузки базы: {e}")


def get_doctor_context(query_text):
    # Поиск документов
    results = vectorstore.similarity_search_with_relevance_scores(
        query_text, 
        k=TOP_K
    )

    if not results:
        print("Результаты не найдены.")
        return ""

    # === ФОРМИРОВАНИЕ DOCTOR_DOC ===
    # Мы создаем список, куда сложим все найденные ответы
    doc_entries = []
    
    # === ИЗМЕНЕННЫЙ УЧАСТОК КОДА ===
    for i, (doc, score) in enumerate(results, 1):
        # В PDF-базе основной текст находится в атрибуте page_content
        pdf_content = doc.page_content
        
        # Также можем вытащить номер страницы, если он есть (PyPDFLoader его создает)
        page_num = doc.metadata.get('page', 'N/A')
        
        # Форматируем блок. Вместо "DOCTOR RESPONSE" пишем "DICTIONARY ENTRY/CONTENT"
        entry = (f"--- RETRIEVED FROM DICTIONARY #{i} (Page: {page_num}, Score: {score:.2f}) ---\n"
                 f"CONTENT:\n{pdf_content}")
        
        doc_entries.append(entry)
    # === КОНЕЦ ИЗМЕНЕННОГО УЧАСТКА ===

    # 4. Склеиваем всё в одну строку переменную DOCTOR_DOC
    # Добавляем заголовок раздела и объединяем кейсы через отступы
    header = "\n\n### REFERENCE DATA FROM MEDICAL DICTIONARY:\n"
    DOCTOR_DOC = header + "\n\n".join(doc_entries)

    return DOCTOR_DOC

Загрузка модели эмбеддингов из: /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_87783/3703796932.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузка векторной базы из: medical_vector_store...


In [2]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Qwen3B_MLX_INT8"
SISTEM = "Cleaned_DATA_QLoRA_with_RAG"
SYSTEM_PROMPT = """
You are an experienced medical doctor providing telemedicine consultation.

CRITICAL GUIDELINES:
1. Base your response on the provided similar medical cases
2. Be professional, empathetic, and evidence-based
3. Provide clear, actionable advice
4. Include warning signs for emergency situations
5. Always recommend consulting with a healthcare provider
6. Never diagnose definitively without examination

RESPONSE STRUCTURE:
- Acknowledge the patient's concern
- Reference similar cases if relevant
- Provide practical guidance
- List red flags requiring immediate attention
- Suggest next steps

DISCLAIMER: This is informational only, not medical advice.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# for key, item in data.items():
#     query_text = item.get("Query", "").strip()

# Для тестирования:
for key, item in list(data.items())[:50]:
    query_text = item.get("Query", "").strip()

    # if not query_text:
    #     print(f"[{key}] SKIPPED (empty query)\n")
    #     continue

    # RAG - логика
    DOCTOR_DOC = get_doctor_context(query_text)
    RAG_content = SYSTEM_PROMPT + DOCTOR_DOC

    messages = [
        {"role": "system", "content": RAG_content},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

The tokenizer you are loading from 'Qwen3B_MLX_INT8' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Done! Results saved to ./evaluation_sistems.json


# LoRA + Prompt + RAG

In [1]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TOP_K = 3

# Один раз загружаю модель эмбеддинга и Векторную Базу Данных
print(f"Загрузка модели эмбеддингов из: {LOCAL_MODEL_DIR}...")
embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=False
    )

print(f"Загрузка векторной базы из: {DB_DIR}...")
try:
        vectorstore = FAISS.load_local(
            DB_DIR,
            embeddings,
            allow_dangerous_deserialization=True,
        )
except Exception as e:
        print(f"Ошибка загрузки базы: {e}")


def get_doctor_context(query_text):
    # Поиск документов
    results = vectorstore.similarity_search_with_relevance_scores(
        query_text, 
        k=TOP_K
    )

    if not results:
        print("Результаты не найдены.")
        return ""

    # === ФОРМИРОВАНИЕ DOCTOR_DOC ===
    # Мы создаем список, куда сложим все найденные ответы врачей
    doc_entries = []
    
    for i, (doc, score) in enumerate(results, 1):
        # 1. Извлекаем ответ врача из метаданных
        # Используем .get() на случай, если ключа вдруг нет, чтобы код не упал
        doctor_response = doc.metadata.get('response', 'No response available in metadata.')
        
        # 2. Форматируем отдельный блок для контекста
        entry = f"--- RETRIEVED MEDICAL CASE #{i} (Score: {score:.2f}) ---\nDOCTOR RESPONSE:\n{doctor_response}"
        doc_entries.append(entry)

    # 4. Склеиваем всё в одну строку переменную DOCTOR_DOC
    # Добавляем заголовок раздела и объединяем кейсы через отступы
    header = "\n\n### REFERENCE CASES FROM MEDICAL DATABASE:\n"
    DOCTOR_DOC = header + "\n\n".join(doc_entries)

    return DOCTOR_DOC

Загрузка модели эмбеддингов из: /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_90042/1566823521.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузка векторной базы из: medical_vector_store...


In [2]:
import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
INPUT_JSON_PATH = "./evaluation_sistems.json"
OUTPUT_JSON_PATH = "./evaluation_sistems.json"
MODEL_PATH = "./Qwen3B_MLX_INT8"
SISTEM = "Cleaned_DATA_QLoRA_with_system_prompt_with_RAG"
SYSTEM_PROMPT = """
You are an AI doctor. Your goal is to write answers that look very similar
in style and format to the reference answers in this dataset.

EXAMPLE OF STYLE:
"Hi, Thanks for the query. Usually posterior subaerial fibroid may not affect the chances of pregnancy.
But it can grow during pregnancy and can lead to complications like red degeneration, difficulty in labor,
increasing the chances of surgical procedures etc. So, better to get treated for the fibroid before planning
for pregnancy. The exact time taken for the fibroid to grow cannot be told as that can differ from person to
person and depends on hormonal levels. Treatment options of fibroid depends on size, symptoms etc. Consult
local gynecologist once and take her opinion. You can go for hormonal pills for few months, that can decrease
the size of the fibroid. If fibroid is not decreasing or increases in size, you may need surgical procedure.
Mastectomy can be done either by laparoscopic method or by laparotomy. After the recovery with your doctors'
advice you can plan for pregnancy. Take care."

INSTRUCTIONS FOR YOUR ANSWER:
- Follow the same tone: polite, explanatory, slightly repetitive, focused on reassurance and guidance.
- Use similar sentence length and paragraph style (continuous prose, no bullet points).
- Mention common phrases like "depends on", "consult local doctor", "take care" when appropriate.
- Adapt the medical content to the new question, but keep the overall style very close to the example.
- Do NOT write lists or headings; use only plain sentences.
- Do NOT refer to this example explicitly in your answer.
"""
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

for key, item in data.items():
    query_text = item.get("Query", "").strip()

# # Для тестирования:
# for key, item in list(data.items())[:1]:
#     query_text = item.get("Query", "").strip()

    if not query_text:
        print(f"[{key}] SKIPPED (empty query)\n")
        continue

    # RAG - логика
    DOCTOR_DOC = get_doctor_context(query_text)
    RAG_content = SYSTEM_PROMPT + DOCTOR_DOC

    messages = [
        {"role": "system", "content": RAG_content},
        {"role": "user", "content": query_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    stdout_buffer = io.StringIO()
    start_time = time.time()

    with redirect_stdout(stdout_buffer):
        response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # Нам нужен verbose для парсинга токенов
        )

    latency_sec = round(time.time() - start_time, 4)
    stdout_text = stdout_buffer.getvalue()

    # --- PARSE TOKENS ---
    prompt_tokens = 0
    generated_tokens = 0
    
    # Извлекаем числа из строк вида "Prompt: 45 tokens" и "Generation: 120 tokens"
    p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
    g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
    if p_match: prompt_tokens = int(p_match.group(1))
    if g_match: generated_tokens = int(g_match.group(1))
    total_tokens = prompt_tokens + generated_tokens

    # --- UPDATE (NOT OVERWRITE) ---
    # Используем .update(), чтобы не удалить ключи для оценки (Clinical correctness и т.д.)
    item[SISTEM].update({
        "output": response.strip(),
        "latency_sec": latency_sec,
        "prompt_tokens": prompt_tokens,
        "generated_tokens": generated_tokens,
        "total_tokens": total_tokens
    })


# --- FINAL SAVE ---
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Done! Results saved to {OUTPUT_JSON_PATH}")

The tokenizer you are loading from 'Qwen3B_MLX_INT8' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Done! Results saved to ./evaluation_sistems.json


# Анализ оценок от LLM-as-a-Judge (Chat_GPT 5.2 и Gemini 3 Thinking)

**Формирую таблицы для вывода метрик от судей**

In [ ]:
import json
import numpy as np
import pandas as pd
from scipy import stats
import os

# 1. Анализ двух оценок от LLM-as-a-Judge (Chat_GPT 5.2 и Gemini 3 Thinking)

INPUT_FILES = [
    'Judge_GPT.json', 
    'Judge_Gemini.json'
]

TARGET_MODELS = {
    "Doctor_answer": "Doctor",
    "Base_model_Qwen3B": "Base_model_Qwen3B",
    "Base_model_Qwen3B_with_system_prompt": "Base_model_Qwen3B_with_system_prompt",
    "Base_model_Qwen3B_with_RAG": "Base_model_Qwen3B_with_RAG",
    "Base_model_Qwen3B_with_system_prompt_with_RAG": "Base_model_Qwen3B_with_system_prompt_with_RAG",
    "Cleaned_DATA_QLoRA": "Cleaned_DATA_QLoRA",
    "Cleaned_DATA_QLoRA_with_system_prompt": "Cleaned_DATA_QLoRA_with_system_prompt",
    "Cleaned_DATA_QLoRA_with_RAG": "Cleaned_DATA_QLoRA_with_RAG",
    "Cleaned_DATA_QLoRA_with_system_prompt_with_RAG": "Cleaned_DATA_QLoRA_with_system_prompt_with_RAG"
}

METRICS = [
    "latency_sec",
    "prompt_tokens",
    "generated_tokens",
    "total_tokens",
    "Clinical correctness & factual accuracy",
    "Appropriate triage & urgency guidance",
    "Safety / harm minimization",
    "Differential diagnosis quality (breadth + plausibility)",
    "Actionability & clarity of plan",
    "Red flags & symptom checking",
    "Communication quality (empathy + tone + readability)"
]

def calculate_confidence_interval(data, confidence=0.95):
    """Считает среднее и размах доверительного интервала."""
    a = np.array(data, dtype=float)
    n = len(a)
    if n < 2:
        return np.mean(a) if n == 1 else 0, 0
    
    m = np.mean(a)
    se = stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

def main():
    # Единое хранилище для всех файлов
    storage = {model: {metric: [] for metric in METRICS} for model in TARGET_MODELS.keys()}
    
    processed_files_count = 0

    # 2. СБОР ДАННЫХ ИЗ НЕСКОЛЬКИХ ФАЙЛОВ
    for file_path in INPUT_FILES:
        if not os.path.exists(file_path):
            print(f"Предупреждение: Файл {file_path} не найден и будет пропущен.")
            continue
            
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                processed_files_count += 1
                
                # Парсинг текущего файла
                for item_id, content in data.items():
                    for display_name, json_key in TARGET_MODELS.items():
                        if json_key in content:
                            block_data = content[json_key]
                            for metric in METRICS:
                                if metric in block_data:
                                    val = block_data[metric]
                                    if isinstance(val, (int, float)):
                                        # Добавляем в общий котел
                                        storage[display_name][metric].append(val)
                                        
        except json.JSONDecodeError:
            print(f"Ошибка: Не удалось прочитать JSON из файла {file_path}.")

    if processed_files_count == 0:
        print("Не обработано ни одного файла. Завершение.")
        return

    print(f"Успешно обработано файлов: {processed_files_count}")

    # 3. ВЫВОД (СГРУППИРОВАНО ПО МЕТРИКАМ)
    results = []
    
    print(f"\n{'Metric':<55} | {'Model':<50} | {'Mean ± 95% CI':<20} | {'N':<5}")
    print("=" * 140)

    for metric in METRICS:
        has_data_for_metric = False
        
        for model in TARGET_MODELS.keys():
            values = storage[model][metric]
            
            if not values:
                continue 

            has_data_for_metric = True
            mean_val, ci_val = calculate_confidence_interval(values)
            count = len(values)
            
            res_string = f"{mean_val:.2f} ± {ci_val:.2f}"
            
            print(f"{metric:<55} | {model:<50} | {res_string:<20} | {count:<5}")
            
            results.append({
                "Metric": metric,
                "Model": model,
                "Mean": mean_val,
                "CI_95": ci_val,
                "Formatted": res_string,
                "Count": count
            })
        
        if has_data_for_metric:
            print("-" * 140)

    # 4. СОХРАНЕНИЕ
    if results:
        df = pd.DataFrame(results)
        df = df[["Metric", "Model", "Mean", "CI_95", "Formatted", "Count"]]
        
        output_csv = "V2combined_metrics_summary.csv"
        df.to_csv(output_csv, index=False)
        print(f"\nСгруппированный отчет по {processed_files_count} файлам сохранен в: {output_csv}")

if __name__ == "__main__":
    main()

Успешно обработано файлов: 2

Metric                                                  | Model                                              | Mean ± 95% CI        | N    
latency_sec                                             | Base_model_Qwen3B                                  | 7.79 ± 0.44          | 84   
latency_sec                                             | Base_model_Qwen3B_with_system_prompt               | 8.01 ± 0.32          | 84   
latency_sec                                             | Base_model_Qwen3B_with_RAG                         | 13.54 ± 0.56         | 84   
latency_sec                                             | Base_model_Qwen3B_with_system_prompt_with_RAG      | 10.13 ± 0.45         | 84   
latency_sec                                             | Cleaned_DATA_QLoRA                                 | 2.38 ± 0.12          | 84   
latency_sec                                             | Cleaned_DATA_QLoRA_with_system_prompt              | 4.44 ± 0.70        